In [1]:
import numpy as np
import pandas as pd
import ast
import os
import sys
from tensorflow.keras.models import Sequential
from keras.layers import Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Dense, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras import backend as K
import itertools
RMSProp = keras.optimizers.RMSprop 

2022-11-17 19:54:18.896366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 19:54:19.070558: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-17 19:54:19.074599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 19:54:19.074613: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
# label = np.load('./img_label.npy',allow_pickle=True)
# dataset = np.load('./img_data.npy')

In [3]:
mal_fam_cont = {'mirai': 0, 'unknown': 0, 'penguin': 0, 'fakebank': 0, 'fakeinst': 0, 'mobidash': 0, 'berbew': 0, 'wroba': 0}

In [4]:
# label = list(label)
# dataset = list(dataset)

In [5]:
# # 10 each for 'mirai','unknown','penguin','fakebank', 'fakeinst', 'mobidash', 'berbew', 'wroba' 
# sub_dataset = []
# sub_label = []
# for i in range(len(label)):
#     if mal_fam_cont[label[i]] < 10:
#         sub_dataset.append(dataset[i])
#         sub_label.append(label[i])
#         mal_fam_cont[label[i]] += 1

In [6]:
# mal_fam_cont

In [7]:
dataset = pd.read_csv(r'./../Labels_TimeSyscallSeqs.csv')

In [8]:
new_dataset = []
# drop_fam = ['metasploit','zbot','skeeyah','hiddad']
drop_index = []
dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
for i in range(len(dataset)):
    if dataset['FAM'][i] in mal_fam_cont and mal_fam_cont[dataset['FAM'][i]] < 10:
        mal_fam_cont[dataset['FAM'][i]] += 1
    else:
        drop_index.append(i)
        
dataset = dataset.drop(drop_index)

In [9]:
dataset = dataset.reset_index(drop = True)

In [10]:
dataset

,FILENAME,FAM,SEQUENCE
0,ca970cc75ccdf7bd1959b970de1f81d682ea3af871aff6...,mirai,"[('1659223215.715424', 'execve'), ('1659223215..."
1,699ffec8df22106524defd950bff835735051514a9b059...,mirai,"[('1662079445.156731', 'execve'), ('1662079445..."
2,94e883f37be5e6a2791b081e78c86804b427e90e55a172...,mirai,"[('946684902.916742', 'execve'), ('946684903.7..."
3,6b8490cc0ea6b234de8b6529b9277ee0ccba22026b4689...,unknown,"[('1970.5854504280915968', 'wait4'), ('1970.62..."
4,93a9669eea9b49258a1c4ca20b261561b6bd91b43643ad...,mirai,"[('1970.-1293549660274688', 'read'), ('1970.-1..."
...,...,...,...
75,94a85f288f8e88b821b61bcb1caa08c1f38230c9da0a11...,penguin,"[('1970.000000', 'execve'), ('2022.064030', 'c..."
76,814e3aa1aec837c9d9e4e2ae03f66cd821fb648237cf69...,penguin,"[('1970.000000', 'execve'), ('2022.052163', 'b..."
77,334a894d9e7e7d8b3869212e5a77b2a897b6a7171b5f4e...,penguin,"[('1970.000000', 'execve'), ('2022.051174', 'n..."
78,b468d1f8084fbefa850888d87e42ad5f30079547b66823...,penguin,"[('1970.000000', 'execve'), ('2022.068917', 'b..."


In [11]:
mal_fam_label = {'mirai': 0, 'unknown': 1, 'penguin': 2, 'fakebank': 3, 'fakeinst': 4, 'mobidash': 5, 'berbew': 6, 'wroba': 7}
label = []
for fam in dataset.FAM:
    label.append(mal_fam_label[fam])

In [12]:
def ExtractSyscallSeq(SeqStr):
    SeqStr = ast.literal_eval(SeqStr)
    return [syscall for time, syscall in SeqStr]

In [13]:
Seq = list()
Seq = [ ExtractSyscallSeq(seq) for seq in dataset.SEQUENCE ]

In [14]:
max_len = 0
for i in range(80):
    max_len = max(max_len, len(Seq[i]))

In [15]:
max_len

5332

In [16]:
#ngram word2vec
from gensim.models import Phrases

bigram_transformer = Phrases(Seq)

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100 #40

model = Word2Vec(bigram_transformer[Seq], min_count = 1)
word_dict = dict(zip(model.wv.index_to_key, model.wv.vectors))

tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(Seq)
X = tokenizer.texts_to_sequences(Seq)
word_index = tokenizer.word_index

# padding, cuz the len of each sample is different
X = pad_sequences(X, maxlen = MAX_SEQUENCE_LENGTH)

#check how many words we use
num_words = min(MAX_NB_WORDS, len(word_dict) + 1) 

# set embedding matrix

num_words = len(word_dict)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for i,w_v in enumerate(word_dict.items()):
    embedding_matrix[i + 1] = w_v[1]

# matrix -> layer
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [17]:
# weights = torch.FloatTensor(model.wv.vectors)
# # weights=[embedding_matrix]
# embedding = nn.Embedding.from_pretrained(weights)

In [18]:
# embedding

In [19]:
len(word_dict)

62

In [20]:
# word_dict

In [21]:
# set the support set

support_set = np.empty([4,100], dtype=np.int_)
count=0
for i in range(40):
    if i%10==0:
        support_set[count]=X[i]
        count+=1

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, label, test_size=0.3)

In [23]:
X.shape

(80, 100)

In [24]:
num = { 0: 0, 1: 0, 2: 0, 3: 0, 4 : 0, 5: 0, 6: 0, 7: 0}
for i in Y_train:
    num[i] += 1
num

{0: 8, 1: 9, 2: 6, 3: 5, 4: 7, 5: 7, 6: 7, 7: 7}

In [25]:
# from xgboost import XGBClassifier

# # 建立 XGBClassifier 模型
# xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.5)
# # 使用訓練資料訓練模型
# xgboostModel.fit(X_train, Y_train)
# # 使用訓練資料預測分類
# predicted = xgboostModel.predict(X_train)

In [26]:
# # 預測成功的比例
# print('訓練集: ',xgboostModel.score(X_train,Y_train))
# print('測試集: ',xgboostModel.score(X_test,Y_test))
# 訓練集:  1.0
# 測試集:  0.20833333333333334

In [27]:
# def the func -> make pair
# make pair
import random
def make_pairs(x, y):
    pairs = []
    labels = []
    for i in range(5):
        for idx1 in range(len(x)):
            x1 = x[idx1]
            label1 = int(y[idx1])
            idx2=random.randint(0,len(x)-1)
            while(idx1 == idx2 or int(y[idx2])==label1):
                idx2=random.randint(0,len(x)-1)
            x2 = x[idx2]
            pairs += [[x1, x2]]
            labels +=[0]
            idx2=random.randint(0,len(x)-1)
            while(idx1 == idx2 or int(y[idx2])!=label1):
                idx2=random.randint(0,len(x)-1)
            x2 = x[idx2]
            pairs += [[x1, x2]]
            labels +=[1]
    return np.array(pairs), np.array(labels).astype("float32")

pairs_train, labels_train = make_pairs(X_train, Y_train)
x_train_1 = pairs_train[:, 0] 
x_train_2 = pairs_train[:, 1]

In [28]:
# def siamese network

def siamese_network(input_shape):
    image1 = keras.layers.Input(input_shape)
    image2 = keras.layers.Input(input_shape)  
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(units=64,
                            return_sequences=True),
            input_shape=(MAX_SEQUENCE_LENGTH , 1)
        )
    )
    #model.add(Dropout(0.5))
    model.add(Conv1D(filters=8,kernel_size = 3, padding="same", activation='relu'))
    #model.add(MaxPooling1D(pool_size=3 ,  padding='same')) 
    model.add(Flatten())
    model.add(Dense(units=50,activation='relu'))
    model.add(Dense(units=10,activation='relu'))
    #model.add(Dense(units=count,activation='relu'))
    model.summary()

    #model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    encoded_1 = model(image1)
    encoded_2 = model(image2)   
    lambda_result = keras.layers.Lambda(lambda x: K.sqrt(K.maximum(K.sum(K.square(x[0] - x[1]), axis = 1, keepdims=True), K.epsilon())))([encoded_1, encoded_2])
    return keras.Model(inputs=[image1,image2],outputs=lambda_result)

In [29]:
# def some function and model

def contrastive_loss(y_true, y_pred):
    margin = 1
    sqaure_pred =  K.square(y_pred)
    margin_square =  K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)

def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

batch_size = 2
epochs = 10
input_shape= x_train_1.shape[1:]
train_siamese = siamese_network(input_shape)    
train_siamese.compile(optimizer = RMSProp(), loss = contrastive_loss, metrics=[accuracy],)
history=train_siamese.fit([x_train_1,x_train_2],
                labels_train,
                batch_size= batch_size,
                epochs= epochs,)

2022-11-17 19:54:31.085141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-17 19:54:31.085168: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 19:54:31.085181: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (islab-Reborn1114): /proc/driver/nvidia/version does not exist
2022-11-17 19:54:31.085399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          6300      
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         84480     
 l)                                                              
                                                                 
 conv1d (Conv1D)             (None, 100, 8)            3080      
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 50)                40050     
                                                                 
 dense_1 (Dense)             (None, 10)                510       
                                                        

In [30]:
# X_train.shape

In [31]:
# embedding = nn.Embedding(num_embeddings = 62, embedding_dim = 100, padding_idx = 0)
# print(embedding.weight, "\n")

In [32]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing, metrics
from sklearn.metrics import f1_score
from joblib import dump,load
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import recall_score, precision_score
import networkx as nx
import os
import torch
import torch.nn.functional as F

In [40]:
class MLP(torch.nn.Module):   # 继承 torch 的 Module
    def __init__(self):
        super(MLP,self).__init__()    # 
        # 初始化三层神经网络 两个全连接的隐藏层，一个输出层
        # self.emb = torch.nn.Embedding(num_embeddings = 62, embedding_dim = 100,padding_idx = 0)
        self.fc1 = torch.nn.Linear(100,128)  # 第一个隐含层  
        self.fc2 = torch.nn.Linear(128,256)  # 第二个隐含层
        self.fc3 = torch.nn.Linear(256,64)
        self.fc4 = torch.nn.Linear(64,16)
        # self.fc5= torch.nn.Linear(256,256)
        # self.fc6 = torch.nn.Linear(256,64)
        self.fc5= torch.nn.Linear(16,8)   # 输出层
        
    def forward(self,din):
        # 前向传播， 输入值：din, 返回值 dout
        # din = din.view(-1,28*28)       # 将一个多行的Tensor,拼接成一行
        # dout = self.emb(din)
        dout = F.relu(self.fc1(din))   # 使用 relu 激活函数
        dout = F.relu(self.fc2(dout))
        dout = F.relu(self.fc3(dout))
        dout = F.relu(self.fc4(dout))
        # dout = F.relu(self.fc5(dout))
        # dout = F.relu(self.fc6(dout))
        dout = F.softmax(self.fc5(dout), dim = 1)  # 输出层使用 softmax 激活函数
        # 10个数字实际上是10个类别，输出是概率分布，最后选取概率最大的作为预测值输出
        return dout

In [41]:
train_features_init = np.array(X_train)
train_label_init = np.array(Y_train)
train_features_init = torch.from_numpy(train_features_init)
train_label_init = np.array(train_label_init).astype('float32') # 轉換成 nparray
train_label_init = torch.from_numpy(train_label_init).type(torch.LongTensor)


strace_train = torch.utils.data.TensorDataset(train_features_init,train_label_init)

#Testdata
test_features_init = np.array(X_test)
test_label_init = np.array(Y_test)
test_features_init = torch.from_numpy(test_features_init)
test_label_init = np.array(test_label_init).astype('float32') # 轉換成 nparray
test_label_init = torch.from_numpy(test_label_init).type(torch.LongTensor)

strace_test = torch.utils.data.TensorDataset(test_features_init, test_label_init)


In [42]:
n_epochs = 30     # epoch 的数目
batch_size = 1  # 决定每次读取多少图10
train_loader = torch.utils.data.DataLoader(strace_train, batch_size = batch_size, num_workers = 0)
test_loader = torch.utils.data.DataLoader(strace_test, batch_size = batch_size, num_workers = 0)
print(len(train_loader))

56


In [43]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():  # 训练集中不需要反向传播
        for data in test_loader:
            images, labels = data
            outputs = model(images.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
    return 100.0 * correct / total

In [44]:
def train():
    lossfunc = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001)
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data,target in train_loader:
            optimizer.zero_grad()   # 清空上一步的残余更新参数值
            output = model(data.float())    # 得到预测值
            loss = lossfunc(output,target)  # 计算两者的误差
            loss.backward()         # 误差反向传播, 计算参数更新值
            optimizer.step()        # 将参数更新值施加到 net 的 parameters 上
            train_loss += loss.item()*data.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        # 每遍历一遍数据集，测试一下准确率
        test()

In [45]:
model = MLP()

In [46]:
train()

Epoch:  1  	Training Loss: 2.087791
Accuracy of the network on the test images: 16 %
Epoch:  2  	Training Loss: 2.054752
Accuracy of the network on the test images: 20 %
Epoch:  3  	Training Loss: 1.983831
Accuracy of the network on the test images: 20 %
Epoch:  4  	Training Loss: 1.904745
Accuracy of the network on the test images: 25 %
Epoch:  5  	Training Loss: 1.836408
Accuracy of the network on the test images: 16 %
Epoch:  6  	Training Loss: 1.764550
Accuracy of the network on the test images: 29 %
Epoch:  7  	Training Loss: 1.742721
Accuracy of the network on the test images: 20 %
Epoch:  8  	Training Loss: 1.797265
Accuracy of the network on the test images: 25 %
Epoch:  9  	Training Loss: 1.893600
Accuracy of the network on the test images: 12 %
Epoch:  10  	Training Loss: 1.792551
Accuracy of the network on the test images: 20 %
Epoch:  11  	Training Loss: 1.671749
Accuracy of the network on the test images: 20 %
Epoch:  12  	Training Loss: 1.666242
Accuracy of the network on